In [2]:
!pip install groq

In [3]:
!pip install sounddevice scipy ffmpeg-python

In [21]:
import os
import sounddevice as sd
import numpy as np
import ffmpeg
from scipy.io.wavfile import write
from groq import Groq
import time

In [22]:
# Initialize the Groq client
client = Groq()

# Parameters for recording and transcription
duration = 3  # Duration of each recording in seconds
sample_rate = 44100  # Sample rate
wav_filename = "temp_audio.wav"  # Temporary .wav file path
m4a_filename = "temp_audio.m4a"  # Temporary .m4a file path


In [47]:
import requests
import pyttsx3
output_file = "transcription_output.txt"  # File to save all transcriptions
transcriptions = []  # Store all transcription results

print("Starting real-time transcription... Press 'Ctrl+C' to end.")  # Updated to reflect keyboard shortcut for stopping
try:
    while True:
        # Step 1: Record audio for the specified duration
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')
        sd.wait()  # Wait until recording is finished
        
        # Save recorded audio to a temporary .wav file
        wav_filename = "temp_recording.wav"  # Ensure temporary file has a consistent name
        write(wav_filename, sample_rate, audio_data)
        
        # Step 2: Convert .wav to .m4a using ffmpeg
        m4a_filename = "temp_recording.m4a"  # Ensure temporary m4a file has a consistent name
        ffmpeg.input(wav_filename).output(m4a_filename).run(overwrite_output=True)

        # Step 3: Transcribe the .m4a file
        with open(m4a_filename, "rb") as file:
            transcription = client.audio.transcriptions.create(
                file=(m4a_filename, file.read()),  # Audio file for transcription
                model="whisper-large-v3-turbo",
                response_format="json",
                language="en",
                temperature=0.0
            )
            # Get the transcription text
            transcribed_text = transcription.text
            print("Transcription:", transcribed_text)
            transcriptions.append(transcribed_text)  # Append to the list

            # Check for undesired output patterns to stop recording
            if "." or "I don’t understand" in transcribed_text or "I can't understand" in transcribed_text:
                print("Stopping transcription due to unclear audio.")
                break  # Exit the loop

        # Small delay to avoid immediate re-recording
        time.sleep(0.5)

except KeyboardInterrupt:
    print("\nReal-time transcription ended by user.")

finally:
    # Save all transcriptions to a text file
    with open(output_file, "w") as f:
        f.write("\n".join(transcriptions))
    print(f"All transcriptions saved to {output_file}")

    # Cleanup: remove temporary files
    if os.path.exists(wav_filename):
        os.remove(wav_filename)
    if os.path.exists(m4a_filename):
        os.remove(m4a_filename)


from groq import Groq

client = Groq()
# Open and read the content of Yolov_output.txt
with open('Yolov_output.txt', 'r') as file:
    yolov_output_content = file.read()
    

with open('transcription_output.txt', 'r') as file:
    whisper_content = file.read()

# prompt = whisper_content + "\n Context: the user is looking at:" + yolov_output_content
prompt = whisper_content + "/n context: the user is watching a human torso physical model " + yolov_output_content

# print(prompt)
import requests
import re
cleaned_text = prompt.lstrip(' ')
cleaned_text1 = cleaned_text.lstrip(' \t')


# Remove spaces and tabs
cleaned_text1 = cleaned_text.lstrip(' \t')

if "Option" in  cleaned_text1:
    cleaned_text1 = "A"
    
def send_data():
    # URL of the Flask API
    url = "http://195.242.13.130:2025/receive"
    
    # JSON payload to send
    payload = {

        "message": cleaned_text1
    }
    
    # Headers to specify content type
    headers = {
        "Content-Type": "application/json"
    }
    
    # Sending POST request
    response = requests.post(url, json=payload, headers=headers)
    
    # Print the response
    print("Status Code:", response.status_code)
    print("Response JSON:", response.json())
    if response.status_code == 200:
    # Extract the LLaMA response from the JSON
        data = response.json()
        llama_response = data.get("llama_response", "No response from LLaMA")
        print("LLaMA Response: ", llama_response)
    # Convert the text response to speech using pyttsx3
        engine = pyttsx3.init()
        engine.say(data)
        engine.runAndWait()
    else:
        print(f"Error from API: {response.status_code}")
        print(response.text)

if __name__ == "__main__":
    send_data()

# prompt = whisper_content + "/n context: the user is watching a human torso physical model " + yolov_output_content + "/n Also tellparts of it"

# stream = client.chat.completions.create(
#     #
#     # Required parameters
#     #
#     messages=[
#         # Set an optional system message. This sets the behavior of the
#         # assistant and can be used to provide specific instructions for
#         # how it should behave throughout the conversation.
#         {
#             "role": "system",
#             "content": "just tell what user is looking at by context"
#         },
#         # Set a user message for the assistant to respond to.
#         {
#             "role": "user",
#             "content": prompt,
#         }
#     ],

#     # The language model which will generate the completion.
#     model="llama3-8b-8192",

#     #
#     # Optional parameters
#     #

#     # Controls randomness: lowering results in less random completions.
#     # As the temperature approaches zero, the model will become deterministic
#     # and repetitive.
#     temperature=0.5,

#     # The maximum number of tokens to generate. Requests can use up to
#     # 2048 tokens shared between prompt and completion.
#     max_tokens=1024,

#     # Controls diversity via nucleus sampling: 0.5 means half of all
#     # likelihood-weighted options are considered.
#     top_p=1,

#     # A stop sequence is a predefined or user-specified text string that
#     # signals an AI to stop generating content, ensuring its responses
#     # remain focused and concise. Examples include punctuation marks and
#     # markers like "[end]".
#     stop=None,

#     # If set, partial message deltas will be sent.
#     stream=True,
# )

# # Print the incremental deltas returned by the LLM.
# data = []
# for chunk in stream:
#     if hasattr(chunk.choices[0].delta, 'content') and chunk.choices[0].delta.content is not None:
#         text = chunk.choices[0].delta.content
#         data.append(text)
#         print(text, end="", flush=True)
# data = "".join(filter(None, data))


# engine = pyttsx3.init()
# engine.say(data)
# engine.runAndWait()

Starting real-time transcription... Press 'Ctrl+C' to end.
Transcription:  Can you tell me what I am holding?
Stopping transcription due to unclear audio.
All transcriptions saved to transcription_output.txt
Status Code: 200
Response JSON: {'answer': "Question: Can you tell me what I am holding?/n context: the user is watching a human torso physical model Heart\nProvide a detailed educational answer: \n\nThe human torso physical model heart is a visual aid used to illustrate the anatomy of the heart. It is a three-dimensional representation of the heart, typically made of plastic or foam, and is designed to be held in the user's hand. The model heart is usually a simplified representation of the heart, showing the main chambers and structures, such as the atria, ventricles, and septum.\n\nWhen holding the model heart, you can observe the following features:\n\n* The atria are the upper chambers of the heart, which receive blood from the veins. They are typically represented by the two 

RuntimeError: run loop already started